## Create new varible Holidays for Schleswig-Holstein
### Add a variable to identify public holidays in Schleswig-Holstein for the dataset period.


In [1]:
import pandas as pd
from datetime import datetime

# --- Define Holidays for Schleswig-Holstein (2013-2018) ---
# This creates a list of public holidays in Schleswig-Holstein for all years in the dataset
# These dates are used to identify holidays in the dataset

# Fixed holidays (same date every year)
def get_fixed_holidays(years):
    """Generate fixed holidays for given years"""
    fixed = []
    for year in years:
        fixed.extend([
            pd.Timestamp(f'{year}-01-01'),  # New Year's Day
            pd.Timestamp(f'{year}-05-01'),  # Labour Day
            pd.Timestamp(f'{year}-10-03'),  # German Unity Day
            pd.Timestamp(f'{year}-10-31'),  # Reformation Day
            pd.Timestamp(f'{year}-11-01'),  # All Saints' Day
            pd.Timestamp(f'{year}-12-25'),  # Christmas Day
            pd.Timestamp(f'{year}-12-26'),  # Boxing Day
        ])
    return fixed

# Easter-based holidays (moveable dates)
# These are calculated based on Easter Sunday
easter_dates = {
    2013: pd.Timestamp('2013-03-31'),
    2014: pd.Timestamp('2014-04-20'),
    2015: pd.Timestamp('2015-04-05'),
    2016: pd.Timestamp('2016-03-27'),
    2017: pd.Timestamp('2017-04-16'),
    2018: pd.Timestamp('2018-04-01'),
}

def get_easter_holidays(easter_dates):
    """Generate Easter-based holidays from Easter Sunday dates"""
    easter_holidays = []
    for year, easter in easter_dates.items():
        # Good Friday (2 days before Easter)
        easter_holidays.append(easter - pd.Timedelta(days=2))
        # Easter Sunday
        easter_holidays.append(easter)
        # Easter Monday (1 day after Easter)
        easter_holidays.append(easter + pd.Timedelta(days=1))
        # Ascension Day (39 days after Easter)
        easter_holidays.append(easter + pd.Timedelta(days=39))
        # Whit Sunday (49 days after Easter)
        easter_holidays.append(easter + pd.Timedelta(days=49))
        # Whit Monday (50 days after Easter)
        easter_holidays.append(easter + pd.Timedelta(days=50))
        # Corpus Christi (60 days after Easter)
        easter_holidays.append(easter + pd.Timedelta(days=60))
    return easter_holidays

# Carnival Tuesday (moveable, 47 days before Easter)
carnival_dates = {
    2013: pd.Timestamp('2013-02-12'),
    2014: pd.Timestamp('2014-03-04'),
    2015: pd.Timestamp('2015-02-17'),
    2016: pd.Timestamp('2016-02-09'),
    2017: pd.Timestamp('2017-02-28'),
    2018: pd.Timestamp('2018-02-13'),
}

years = [2013, 2014, 2015, 2016, 2017, 2018]

# Combine all holidays
schleswig_holstein_holidays = (
    get_fixed_holidays(years) + 
    get_easter_holidays(easter_dates) + 
    list(carnival_dates.values())
)

# Remove duplicates and sort
schleswig_holstein_holidays = sorted(list(set(schleswig_holstein_holidays)))

print(f"Defined {len(schleswig_holstein_holidays)} holidays for Schleswig-Holstein (2013-2018)")
print("Year distribution:")
for year in years:
    holidays_in_year = sum(1 for h in schleswig_holstein_holidays if h.year == year)
    print(f"  {year}: {holidays_in_year} holidays")


Defined 90 holidays for Schleswig-Holstein (2013-2018)
Year distribution:
  2013: 15 holidays
  2014: 15 holidays
  2015: 15 holidays
  2016: 15 holidays
  2017: 15 holidays
  2018: 15 holidays


### Create a function to add a holiday indicator to the merged dataset.


In [2]:
def add_holiday_indicator(df, holidays, date_column='Datum'):
    """
    Add a holiday indicator column to a DataFrame.
    
    Parameters:
    -----------
    df : pd.DataFrame
        The DataFrame to which the holiday indicator will be added
    holidays : list
        List of pd.Timestamp objects representing holidays
    date_column : str
        The name of the date column in the DataFrame (default: 'Datum')
    
    Returns:
    --------
    pd.DataFrame
        DataFrame with a new 'is_holiday' column (1 if holiday, 0 otherwise)
    """
    df = df.copy()
    # Create a boolean column checking if each date is in the holidays list
    df['is_holiday'] = df[date_column].isin(holidays).astype(int)
    return df

print("Holiday indicator function defined successfully")
print("This function will add an 'is_holiday' column to the merged dataset")


Holiday indicator function defined successfully
This function will add an 'is_holiday' column to the merged dataset


# This code merges the given CSV Datasets and can be used to combine new datasets to it.

## Load and Merge Datasets (CAREFUL: This cell creats a new csv file)
Now proceed with loading the CSV files and performing the outer join. The holiday indicator will be added after the merge.


In [ ]:
import pandas as pd
from pathlib import Path

# --- 1. Define File Names (repo-root-relative paths so nbconvert works) ---
file_umsatz = '1_DatasetCharacteristics/raw_data/umsatzdaten_gekuerzt.csv'
file_test = '1_DatasetCharacteristics/raw_data/test.csv'
file_kiwo = '1_DatasetCharacteristics/raw_data/kiwo.csv'
file_wetter = '1_DatasetCharacteristics/raw_data/wetter.csv'
file_niederschlag = '1_DatasetCharacteristics/raw_data/Niederschlag.csv'

try:
    # --- 2. Load Umsatz and Test, then concatenate them ---
    print(f"Loading {file_umsatz}...")
    df_umsatz = pd.read_csv(file_umsatz, parse_dates=['Datum'])

    print(f"Loading {file_test}...")
    df_test = pd.read_csv(file_test, parse_dates=['Datum'])

    # Concatenate Umsatz and test rows first (append test rows to umsatz)
    df_umsatz_combined = pd.concat([df_umsatz, df_test], ignore_index=True)
    print(f"Combined umsatz + test: {len(df_umsatz_combined)} rows")

    # --- 3. Load other CSVs ---
    print(f"Loading {file_kiwo}...")
    df_kiwo = pd.read_csv(file_kiwo, parse_dates=['Datum'])

    print(f"Loading {file_wetter}...")
    df_wetter = pd.read_csv(file_wetter, parse_dates=['Datum'])

    print(f"Loading {file_niederschlag}...")
    df_niederschlag = pd.read_csv(file_niederschlag, parse_dates=['Datum'])

    # --- 4. Set 'Datum' as the Index for All DataFrames ---
    df_umsatz_combined = df_umsatz_combined.set_index('Datum')
    df_kiwo = df_kiwo.set_index('Datum')
    df_wetter = df_wetter.set_index('Datum')
    df_niederschlag = df_niederschlag.set_index('Datum')

    print("All files loaded and indexed on 'Datum'.")

    # --- 5. Perform the Outer Join ---
    other_dfs = [df_kiwo, df_wetter, df_niederschlag]
    print("Performing outer join with umsatz+test as base...")
    final_df = df_umsatz_combined.join(other_dfs, how='outer')

    # --- 6. Clean Up and Add Holiday Indicator ---
    final_df = final_df.reset_index()
    final_df = final_df.sort_values(by='Datum')
    final_df = add_holiday_indicator(final_df, schleswig_holstein_holidays, date_column='Datum')
    print("Added holiday indicator variable to merged dataset")

    # --- 7. Display Results and Save to File ---
    print("\n--- Join Complete ---")
    print("First 5 rows of the combined data:")
    print(final_df.head())

    print("\nLast 5 rows of the combined data:")
    print(final_df.tail())

    print(f"\nTotal rows in combined file: {len(final_df)}")
    print(f"Holidays found in dataset: {final_df['is_holiday'].sum()}")

    output_filename = 'combined_data_outer.csv'
    final_df.to_csv(output_filename, index=False)
    print(f"\nSuccessfully saved combined data to '{output_filename}'")

except FileNotFoundError as e:
    print(f"\nError: File not found.")
    print(f"Details: {e}")
    print("Please make sure all CSV files are present in '1_DatasetCharacteristics/raw_data/'.")
except Exception as e:
    print(f"\nAn unexpected error occurred: {e}")


Loading ../raw_data/umsatzdaten_gekuerzt.csv...
Loading ../raw_data/test.csv...
Combined umsatz + test: 11164 rows
Loading ../raw_data/kiwo.csv...
Loading ../raw_data/wetter.csv...
Loading ../raw_data/Niederschlag.csv...
All files loaded and indexed on 'Datum'.
Performing outer join with umsatz+test as base...
Added holiday indicator variable to merged dataset

--- Join Complete ---
First 5 rows of the combined data:
       Datum  id  Warengruppe  Umsatz  KielerWoche  Bewoelkung  Temperatur  \
0 2012-01-01 NaN          NaN     NaN          NaN         8.0      9.8250   
1 2012-01-02 NaN          NaN     NaN          NaN         7.0      7.4375   
2 2012-01-03 NaN          NaN     NaN          NaN         8.0      5.5375   
3 2012-01-04 NaN          NaN     NaN          NaN         4.0      5.6875   
4 2012-01-05 NaN          NaN     NaN          NaN         6.0      5.3000   

   Windgeschwindigkeit  Wettercode  Niederschlag  is_holiday  
0                 14.0        58.0          14.

## Merge test.csv to the Dataset
Append the test data to the merged dataset for extended coverage.


In [4]:
try:
    # The merged file was created in the previous cell: 'combined_data_outer.csv'
    output_filename = 'combined_data_outer.csv'
    print(f"Loading merged file '{output_filename}'...")
    df_combined = pd.read_csv(output_filename, parse_dates=['Datum'])
    print(f"Merged file loaded: {len(df_combined)} rows")
    print("Date range:", df_combined['Datum'].min(), "to", df_combined['Datum'].max())
    if 'is_holiday' in df_combined.columns:
        print(f"Holidays in merged data: {df_combined['is_holiday'].sum()}")
    else:
        print("Holiday indicator column not present in merged file.")
except Exception as e:
    print(f"Error loading merged file: {e}")


Loading merged file 'combined_data_outer.csv'...
Merged file loaded: 11782 rows
Date range: 2012-01-01 00:00:00 to 2019-12-31 00:00:00
Holidays in merged data: 294
